# Intro TensorFlow avec tf.keras

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/thalitadru/CoursNNDL/blob/master/TFKerasIntro.ipynb)

tf.keras est l'API haut-niveau par défaut de Tensorflow. Pour la plupart des projets, elle sera souvent suffisante pour exprimer vos modèles, avec l'avantage d'éliminer pas mal de code répétitif "boiler-plate" avec pas mal d'abstractions sur les solveurs d'optimisation (`optimisers`) et sur boucles d'entraînement et validation (`model.fit()` et `model.evaluate()`).
Il s'agit en plus d'une API bien documentée et réputée pour sa facilité de prise en main.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras import layers, losses, optimizers

SEED = 0
np.random.seed(SEED)
rng = np.random.RandomState(SEED)
tf.random.get_global_generator().reset_from_seed(SEED)
tfseed = tf.random.get_global_generator().reset_from_seed

## Modèle `Sequential `

Avec `tf.keras.Sequential` on peut construire des modèles par empilement de couches (`layers`). Elle est adapté à tout modèle qui reçoit un seul tenseur en entrée et renvoie un unique tenseur en sortie.

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Input(shape=(8,)),
        keras.layers.Dense(
            units=4,
            activation="relu",
            name="layer1",
        ),
        keras.layers.Dense(
            3,
            activation="relu",
            name="layer2",
        ),
        keras.layers.Dense(
            1,
            name="layer3",
        ),
    ]
)
model

In [ ]:
model.summary()

In [ ]:
!pip install visualkeras
import visualkeras as vk

vk.layered_view(model, legend=True)

In [ ]:
keras.utils.plot_model(model)

In [ ]:
x = tf.reshape(tf.range(0, 16), shape=(2, -1))
x

In [ ]:
y = model(x)
y

Un modèle `Sequential` n'est pas approprié si:

- Votre modèle (ou une de ses couches) a plusieurs entrées et ou sorties
- Vous désirez partager des poids entre des couches (_layer sharing_)
- Vous souhaitez une topologie non-linéaire (connections qui sautent des couches comme dans les ResNets ou chemins parallèles comme dans les Inception)

Vous devez alors faire appel à l'API fonctionnelle.

### Parenthese: Controle de l'initialization
Afin de pouvoir réproduire les mêmes résultats, je redéclare le modèle ici en precisant la seed et le mode d'initialisation utilisé pour chaque couche.

In [ ]:
# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Input(shape=(8,)),
        layers.BatchNormalization(),
        keras.layers.Dense(
            units=4,
            activation="relu",
            kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
            name="layer1",
        ),
        keras.layers.Dense(
            3,
            activation="relu",
            kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
            name="layer2",
        ),
        keras.layers.Dense(
            1,
            kernel_initializer=tf.keras.initializers.GlorotNormal(seed=SEED),
            name="layer3",
        ),
    ]
)
model

## Avant l'entrainement: `compile`

Avant pouvoir appeler `fit`, il faut effectuer l'étape `compile`.
Pour cela on doit choisir une fonction de cout `loss`

De plus, on peut aussi choisir:
- `optimizer`: un algorithme de descente du gradient: SGD, Adam, RMSprop, etc., et son taux d'apprentissage `learning_rate`
- `metrics`: des métriques additionnelles à calculer

Si on n'informe rien, des valeurs par défaut seront utilisées.

In [ ]:
model.compile(
    optimizer=optimizers.Adagrad(learning_rate=0.001),
    loss=tf.keras.losses.MeanSquaredError(),
)
model.save_weights("init.h5")

## Fit

### Exemple : NN sur California Housing data
 Le modèle qu'on vient de crèer :
 - prends 8 attributs en entrée
 - renvoi une valuer réele en sortie
 
Il correspond aux dimensions du dataset CaliforniaHousing que vous avez utilisé lors du TD précédent.
Chargez ce dataset ici et entraînez ce modèle à l'aide de sa méthode `fit`.
Donnez en entrée :
- les données `X`,
- les targets `y`,
- un nombre d'époques `epochs`,
- et choisissez une fraction des données pour le `validation_split`.


In [ ]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

In [ ]:
# chargez le dataset ici
# fetch_california_housing
d = fetch_california_housing()
X, y = d.data, d.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=rng)

In [ ]:
model.load_weights("init.h5")
## completez l'appel a fit
out = model.fit(
    X_train,
    y=y_train,
    epochs=5,
    batch_size=1,
    validation_split=0.1,
)

In [ ]:
# ??model1.fit

### Partial fit
Sans réinitialiser le modèle, chaque appel à fit continue l'entraînement du réseau pour le nombre d'époques spécifié.
Chaque appel réalise donc un fitting partiel.

In [ ]:
model.optimizer.learning_rate = 0.1
out2 = model.fit(
    X,
    y=y,
    epochs=10,
    batch_size=64,
    validation_split=0.2,
)

## Evaluate
Regardons le score de notre modèle sur l'ensemble complet d'entraînement et sur celui de test

In [ ]:
model.evaluate(X_train, y_train)

In [ ]:
model.evaluate(X_test, y_test)

## Courbes d'apprentissage

### `History` de l'entraînement

L'appel de `fit` renvoie une structure contenant les valeurs des métriques au cours de l'entraînement.

In [ ]:
out.history.keys()

In [ ]:
out.history["loss"][-5:-1]

In [ ]:
out.history["val_loss"][-5:-1]

In [ ]:
model.fit

Ces courbes sauvegardes dans `history` son des courbes d'apprentissage.
On peut les afficher dans un graphique ainsi:

In [ ]:
plt.plot(out.history["loss"][1:], label="train")
plt.plot(out.history["val_loss"][1:], label="val")
plt.legend()

In [ ]:
def learning_curves(history, **kwargs):
    loss = history.history["loss"]
    val_loss = history.history["val_loss"]

    epochs_range = range(1, len(loss) + 1)

    plt.plot(
        epochs_range,
        history.history["loss"],
        marker="o",
        linestyle="dashed",
        label="Train loss",
        **kwargs
    )
    plt.plot(
        epochs_range,
        history.history["val_loss"],
        marker="o",
        linestyle="dashed",
        label="Valid loss",
        **kwargs
    )
    plt.xlabel("epochs")
    plt.ylabel("loss")
    plt.title("courbe d'apprentisage x époques")
    plt.legend()

In [ ]:
learning_curves(out)

### Courbes d'apprentissage et early stopping
Les courbes d'apprentissage servent a suivre l'entraînement du modèle et voir le moment ou il a donnée le meilleurs score en validation (et donc son meilleur potentiel de généralisation). Meme si l'erreur en validation commence à augmenter en fin d'entrainement, il suffit de sauvegarder des checkpoints du modèle sur le époques ou ce, on peut a la fin reprendre le modèle quand le `val_loss` était le plus bas.

## Callbacks

In [ ]:
?tf.keras.callbacks.EarlyStopping

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    patience=3,
    verbose=1,
    mode="auto",
    restore_best_weights=True,
)

In [ ]:
?tf.keras.callbacks.ReduceLROnPlateau

In [ ]:
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.2,
    patience=5,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=0,
    min_lr=0.001,
)

In [ ]:
nan_stop = tf.keras.callbacks.TerminateOnNaN()

In [ ]:
callbacks = [
    early_stop,
    reduce_lr,
    nan_stop,
]

In [ ]:
model.optimizer.learning_rate = 0.5
model.load_weights("init.h5")
out2 = model.fit(
    X_train,
    y=y_train,
    epochs=100,
    batch_size=X_train.shape[0],  # nombre total d'échantillons
    validation_split=0.2,
    verbose=0,
    callbacks=callbacks,
)

In [ ]:
learning_curves(out2)

### Exercice
Évaluez le modèle sur train et test une nouvelle fois. Est-ce que sa généralisation a amélioré?

# La suite: tutoriel Image Classification
1. Sauvegardez et téléchargez le present notebook sur votre machine;
1. Cliquez [ici](https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/tutorials/images/classification.ipynb)